<a href="https://colab.research.google.com/github/ManuelaUsuga/Analitica-de-Negocios/blob/main/Reto_2_%C3%81rbol_Decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Se procede con las cargas de las librerias de trabajo

In [ ]:
#Librerias Cientificas
import numpy as np
import pandas as pd

#Librerias Árbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Conectar al drive => PARA NO TENER QUE SUBIR EL EXCEL several times
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Procede con la carga de los datos

In [ ]:
nxl='/content/2. CollegeAdmisions_Int_M.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0) #College_Admissions
XDB=XDB.dropna() #Elimina datos vacíos

#Seleccionar las variables que queremos
XD=XDB[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
yd=XDB[['Admitted']]





2. Convertir a una variable categórica numérica

In [ ]:
XD['Gender']=XD['Gender'].astype('category').cat.codes #Para cuando una variable está en letras, pasar a #

yd['Admitted']=yd['Admitted'].astype('category').cat.codes

<ipython-input-9-f3dc0868055b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XD['Gender']=XD['Gender'].astype('category').cat.codes #Para cuando una variable está en letras, pasar a #
<ipython-input-9-f3dc0868055b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yd['Admitted']=yd['Admitted'].astype('category').cat.codes


3. Se procede a la implementación del modelo con un máximo de 4 niveles de análisis.

In [ ]:
mar=DecisionTreeClassifier(criterion='gini', max_depth=4) #Modelo Árbol de Decisión / max depth= 4 variables, cuando son más de 4 es mucho para el modelo(complicado)
mar.fit(XD, yd) #Establezca la relación entre variable de entrada y salida

#Se procede con la gráfica
from pydotplus import graph_from_dot_data #Me crea la gráfica
from sklearn.tree import export_graphviz #Me exporta a PNG, JPEG, TIFF

ve=['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT'] #Nombres para el Árbol
dot_data=export_graphviz(mar, feature_names=ve) #Crear el gráfico
graph=graph_from_dot_data(dot_data) #Entre los dos se hace la gráfica
graph.write_png('2.ÁrbolDecisión_MReto.png')

True

4. Se procede con el pronóstico de los solicitantes que no poseen Admitted

In [ ]:
#Pronóstico para cuando es una hoja aparte
XDB0=pd.read_excel(nxl,sheet_name=1) #Pronóstico

XDB0['Gender']=XDB0['Gender'].astype('category').cat.codes #Para cuando una variable está en letras, pasar a #

#No utilizar dropna porque el punto es encontrar aquellos datos sin aprobación
XDB0.head()

XD0=XDB0[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
yd0=mar.predict(XD0)

#Se crea el informe en excel
df=pd.DataFrame(np.column_stack((XD0,yd0)))
df.columns=['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT', 'Admitted']
df.to_excel("Evaluación Admitidos.xlsx")

5. Evaluar el comportamiento del modelo

In [ ]:
ydp=mar.predict(XD) #Pronóstico sobre la base de datos original
cm=confusion_matrix(yd,ydp)
print(cm)
print("La matriz de confusión es:\n", cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN)
print("La exactitud es: ", Exactitud)

Sensibilidad=VP/(VP+FN)
print("La sensibilidad es: ", Sensibilidad)

Especificidad=VN/(VN+FP)
print("La especificidad es: ", Especificidad)

Precision=VP/(VP+FP)
print("La precisión es: ", Precision)

TasaError=(FP+FN)/(VP+VN+FP+FN)
print("La tasa de error es: ", TasaError)

PredNeg=VN/(VN+FN)
print("La tasa de falsos negativos es: ", PredNeg)

PredPos=VP/(VP+FP)
print("La tasa de falsos positivos es: ", PredPos)

[[  20  205]
 [   2 1301]]
La matriz de confusión es:
 [[  20  205]
 [   2 1301]]
La exactitud es:  0.8645287958115183
La sensibilidad es:  0.9984650805832693
La especificidad es:  0.08888888888888889
La precisión es:  0.8638778220451527
La tasa de error es:  0.1354712041884817
La tasa de falsos negativos es:  0.9090909090909091
La tasa de falsos positivos es:  0.8638778220451527


**Descripción de los datos**

De acuerdo con la base de datos 'College Admissions', esta posse un total de 17.339 datos totales divididos en 12.016 Admitidos y 5.323 No admitidos por la universidad. Cada uno de los aplicantes estará descrito en función de 4 variables:

**Edu_Parent1:** Representa el nivel educativo alcanzado por el primer padre o tutor del aplicante. Se clasifica en:

1: Sin escuela preparatoria.

2: Escuela preparatoria trunca.

3: Graduado de la escuela preparatoria.


**Edu_Parent2:** Indica el nivel educativo alcanzado por el segundo padre o tutor del aplicante. Se clasifica en:

4: Universidad trunca.

6: Graduado universitario en una carrera de 4 años.

7: Posgrado.


**Gender:** Identifica el sexo del aplicante, utilizando las siguientes categorías:

1: Hombre.
0: Mujer.

**White:** Indica si el aplicante se identifica como blanco o no:

1: Blanco.

0: No blanco.


**Asian:** Indica si el aplicante se identifica como asiático o no:

1: Asiático.

0: No asiático.

**HSGPA:** GPA ponderado de la escuela preparatoria, con un rango de 0 a 5. Representa el rendimiento académico del estudiante en la escuela preparatoria y es una variable numérica continua.

**SAT/ACT:** La variable SAT/ACT representa la puntuación más alta obtenida en los exámenes SAT o ACT por cada aplicante. Si el aplicante tomó el ACT en lugar del SAT, su puntuación se convierte a la escala del SAT, específicamente en la puntuación equivalente para las secciones de inglés y matemáticas.


**Análisis de resultados**

Los resultados del modelo de clasificación pueden evaluarse a partir de la matriz de confusión, donde destacan los Verdaderos Negativos (VN) = 1301, que representan los casos correctamente identificados como negativos, y los Verdaderos Positivos (VP) = 20, que corresponden a los casos correctamente clasificados como positivos.

El modelo alcanza una Exactitud del 81%, lo que indica que tiene una buena capacidad predictiva general. Sin embargo, su Sensibilidad del 62% sugiere que el modelo tiene dificultades para identificar correctamente todos los casos positivos.

Por otro lado, su Especificidad del 89% indica que es bastante eficaz en la detección de negativos, con una Precisión del 72%, lo que significa que la mayoría de las predicciones positivas son correctas.

No obstante, la tasa de falsos negativos del 84.38% es alta, lo que indica que el modelo no está identificando una cantidad significativa de casos positivos. A su vez, la tasa de falsos positivos del 72.02% muestra que el error en la predicción de positivos es considerable, lo que podría afectar su utilidad en ciertas aplicaciones.

El total de muestras es 1,528, con una distribución de [225, 1303], lo que indica que 1,303 personas fueron admitidas.

El árbol de decisión cuenta con 7 nodos puros, que son aquellos en los que todos los datos pertenecen a una sola clase y, por lo tanto, su índice de Gini es 0.0.

Un ejemplo de esto es el **nodo con samples = 5, value = [0,5]**, que representa a cinco estudiantes que no fueron admitidos en la universidad. Para llegar a este nodo, el árbol sigue una secuencia de decisiones desde la raíz:

1. Se clasifica a los estudiantes con un HSGPA menor o igual a 2.98, lo que los ubica en la rama izquierda del árbol.
2. Dentro de este grupo, se considera el puntaje SAT/ACT, donde aquellos con una puntuación menor o igual a 1185.0 continúan por otra división.
3. Finalmente, dentro de esta segmentación, los estudiantes con HSGPA menor o igual a 2.92 llegan a un nodo puro donde todas las observaciones pertenecen a la categoría de no admitidos.

Observando el árbol, identificamos los siguientes nodos puros:

Nodo con samples = 1, value = [0,1]

Nodo con samples = 1, value = [0,1]

Nodo con samples = 1, value = [1,0]

Nodo con samples = 5, value = [0,5]

Nodo con samples = 4, value = [4,0]

Nodo con samples = 1, value = [0,1]

Nodo con samples = 69, value = [0,69]

Total de nodos puros: 7.

Estos nodos indican que el árbol de decisión logró clasificar completamente 7 subconjuntos de datos sin ambigüedad.

